In [1]:
#FINAL#


import requests
import json
from datetime import datetime, timedelta
from time import sleep
import csv
import re

In [2]:
#FINAL#
linksurl = "LinksPMQsMasterRolling.csv" 
#rolling database file of Dates that PMQ debates happened and Hansard links to their transcripts

api doc = https://hansard-api.parliament.uk/swagger/ui/index#!/Search/Search_SearchDebates

In [3]:
#FINAL#
baseurl = "https://hansard-api.parliament.uk/search/debates.json?queryParameters.house=Commons&queryParameters.startDate={0}&queryParameters.endDate={0}&queryParameters.searchTerm=Oral%20Answers%20to%20Questions"

In [4]:
#FINAL#

#new links check - checks the ones in the file, and identifies any new wednesdays
with open(linksurl,'r+',newline='') as f:
    f.readline()
    txt = csv.reader(f)
    dateslist = []
    for t in txt:
        dateslist.append(t[1])
    latestd = max([datetime.strptime(x,'%d/%m/%Y') for x in dateslist])
    now = datetime.now()
    tdy = datetime(year=now.year,month=now.month,day=now.day)
    #tdy = datetime(year=2023,month=4,day=22) 
    toscrape = []


#latestd = datetime.strptime('20/09/2021','%d/%m/%Y')
#now = datetime.now()
#tdy = datetime(year=now.year,month=now.month,day=now.day)
#toscrape = []
    while latestd < tdy:
        latestd = latestd + timedelta(1)
        if datetime.strftime(latestd,"%a") == "Wed":
            qdate = datetime.strftime(latestd,"%Y-%m-%d")
            fdate = datetime.strftime(latestd,'%d/%m/%Y')
            
            #scrapeurl = linkscrape(baseurl.format(qdate,qdate),qdate)
            print(baseurl.format(qdate,qdate))
            src = baseurl.format(qdate,qdate)
            _data = requests.get(src)
            sleep(1)
            jase = json.loads(_data.text)
            resies = []
            for j in jase['Results']:
                resies.append(j["DebateSectionExtId"])
            if not resies is None:
                
                toscrape.append([resies,fdate])
            





    wri = csv.writer(f)
    for tt in toscrape:
        wri.writerow(tt)

https://hansard-api.parliament.uk/search/debates.json?queryParameters.house=Commons&queryParameters.startDate=2023-07-12&queryParameters.endDate=2023-07-12&queryParameters.searchTerm=Oral%20Answers%20to%20Questions
https://hansard-api.parliament.uk/search/debates.json?queryParameters.house=Commons&queryParameters.startDate=2023-07-19&queryParameters.endDate=2023-07-19&queryParameters.searchTerm=Oral%20Answers%20to%20Questions
https://hansard-api.parliament.uk/search/debates.json?queryParameters.house=Commons&queryParameters.startDate=2023-07-26&queryParameters.endDate=2023-07-26&queryParameters.searchTerm=Oral%20Answers%20to%20Questions
https://hansard-api.parliament.uk/search/debates.json?queryParameters.house=Commons&queryParameters.startDate=2023-08-02&queryParameters.endDate=2023-08-02&queryParameters.searchTerm=Oral%20Answers%20to%20Questions
https://hansard-api.parliament.uk/search/debates.json?queryParameters.house=Commons&queryParameters.startDate=2023-08-09&queryParameters.end

In [ ]:
#ADHOC CHECK

latestd = datetime.strptime('20/09/2021','%d/%m/%Y')
now = datetime.now()
tdy = datetime(year=now.year,month=now.month,day=now.day)
toscrape = []
while latestd < tdy:
    latestd = latestd + timedelta(1)
    if datetime.strftime(latestd,"%a") == "Wed":
        qdate = datetime.strftime(latestd,"%Y-%m-%d")
        fdate = datetime.strftime(latestd,'%d/%m/%Y')
        
        #scrapeurl = linkscrape(baseurl.format(qdate,qdate),qdate)
        print(baseurl.format(qdate,qdate))
        src = baseurl.format(qdate,qdate)
        _data = requests.get(src)
        sleep(1)
        jase = json.loads(_data.text)
        resies = []
        for j in jase['Results']:
            resies.append(j["DebateSectionExtId"])
        if not resies is None:
            
            toscrape.append([resies,fdate])

In [5]:
toscrape

[[['466F7380-BB52-41DE-A6F8-C8E41EACDD5E'], '12/07/2023'],
 [['715F79EA-262B-4D1A-BCD7-D528F8D1D945'], '19/07/2023'],
 [[], '26/07/2023'],
 [[], '02/08/2023'],
 [[], '09/08/2023'],
 [[], '16/08/2023'],
 [[], '23/08/2023'],
 [[], '30/08/2023'],
 [['16E9A353-DE76-4F58-BC81-36520CBA4DC4'], '06/09/2023'],
 [['0ECAEAAB-EA8A-4F6E-858A-07FEE93907BA'], '13/09/2023'],
 [[], '20/09/2023'],
 [[], '27/09/2023'],
 [[], '04/10/2023'],
 [[], '11/10/2023']]

In [14]:
test = requests.get("https://hansard-api.parliament.uk/search/debates.json?queryParameters.house=Commons&queryParameters.startDate=2021-12-15&queryParameters.endDate=2021-12-15&queryParameters.searchTerm=Oral%20Answers%20to%20Questions")

In [6]:
#FINAL

debateq = "https://hansard-api.parliament.uk/debates/debate/{0}.json"

In [7]:
#FINAL

#toscrape is a list of lists, with each inner list being a debate ID
appendlist = []
for t in toscrape:
    if len(t[0]) == 0:
        print('No debates')
    elif len(t[0]) >= 2:
        print('More than 1 debate with Questions in title - investigate manually')
    else:
        datx = datetime.strptime(t[1],'%d/%m/%Y')
        datf = datetime.strftime(datx,"%Y-%m-%d")
        debx = requests.get(debateq.format(t[0][0]))
        debj = json.loads(debx.text)
        appendlist.append([t[1],debj])

No debates
No debates
No debates
No debates
No debates
No debates
No debates
No debates
No debates
No debates


In [8]:
#FINAL


contribs = []

for app in appendlist:
    dt = app[0]
    jso = app[1]['ChildDebates']
    for cdeb in jso:
        
        if 'prime minister' in cdeb['Overview']['Title'].lower():
            itm = cdeb['Items']
            itms_outside_cdebs = [i for i in itm if i['MemberId'] is not None]
            print(len(itms_outside_cdebs))
            subcdeb = cdeb['ChildDebates']
            data = len(subcdeb)
            print(data)
            if data >1:
                for c in subcdeb:
                    print(dt,c['Overview']['Title'],'\n')
                    
            for cx in subcdeb:
                checksubdebs = len(cx['ChildDebates']) >= 1
                if checksubdebs: print('1')
                itms = [[dt,x['MemberId'],x['AttributedTo'],x['Timecode'],x['Value']] for x in cx['Items'] if x['ItemType'] == 'Contribution']
                #itms = [x for x in cx['Items'] if x['ItemType'] == 'Contribution']
                contribs.extend(itms)
                



#for x in appendlist[0]['ChildDebates']:
#    if 'prime minister' in x['Overview']['Title'].lower():
#        for j in x:
#            if j == 'ChildDebates':
#                #print(j,'---',x[j])
#                for i in x[j]:
#                    for p in i:
#                        if p == 'Items':
#                            #print(i[p])
#                            for m in i[p]:
#                                print(m)
#                        #print(i,'\n\n')

0
2
12/07/2023 Oil Production and Domestic Energy Prices 

12/07/2023 Engagements 

0
1
0
1
0
3
13/09/2023 Engagements 

13/09/2023 West Midlands: Economic Outlook 

13/09/2023 Engagements 



In [9]:
 contribs[-1]
#for c in contribs:
#    if c[1] is not None: print(c)

['13/09/2023',
 467,
 'Mr Speaker',
 None,
 'May I just say thank you to Sir John Benger, in his final Prime Minister’s questions, for his loyal service to the House? We do appreciate it. Thank you, Sir John.']

In [10]:
#FINAL


def cleanup(x):
    
    cleanups_log = []
    straight_with_space = ['\n','\t','\r','(LD):','(Con):','(Lab):','(Ind):','(Lab/Co-op):','\x96',
                           '\x97','\u2060','\u07de','\u2004','\xa0','\u2002','\u2003','\u2014']
    for s in straight_with_space:
        x = x.replace(s,' ')
    straight_w_custom = [['\u1d9c','c'],
                         ['\u011f','g'],['\u0177','y'],['\u0175','w'],['\x93','"'],['\u201c','"'],['\u201d','"'],
                         ['\u2019',"'"],['\x92','"'],['\x94','"'],['\u0103','a'],['\u010d','c']]
    for s in straight_w_custom:
        x = x.replace(s[0],s[1])
        cleanups_log.append([s[0],s[1]])
    
    #regex replacements
    
        # mps names in brackets
    membreg = ['Member for [A-Za-z\s,\-]{1,40}\([A-Z][A-Za-z]{1,20} [A-Z][A-Za-z]{1,20}\)',
                  'Member for [A-Za-z\s,\-]{1,40}\([A-Z][A-Za-z]{1,14} [A-Z][A-Za-z]{1,15} [A-Z][A-Za-z]{1,15}\)']
    xmembreg = '|'.join(membreg)    
    if re.search(xmembreg,x):
        mbmr = re.finditer(xmembreg,x)
        
        for d in mbmr:
            new_d = re.sub('\s\([A-Za-z\s]{1,}\)$','',d[0])
            x = x.replace(d[0],new_d)
            cleanups_log.append([d[0],new_d])        
    
    complexr = ['<span ','data-house-id="[a-zA-Z0-9_\-]{1,}"',
                'data-volume-number="[a-zA-Z0-9_\-]{1,}"',
                'id="[a-zA-Z0-9_\-]{1,}"','class="[a-zA-Z0-9\-]{1,}"',
     'data-column-number="[a-zA-Z0-9\-]{1,}"','</span>',
                '</Question>','</QuestionText>','Q[1-9]{1,2}\. ',
                '<Question HRSContentId="\{[A-Z0-9\-]{1,}\}">','<QuestionText HRSContentId="\{[A-Z0-9\-]{1,}\}">',
                '<em>Interruption.</em>','<em>Interruption</em>','\[Interruption\]','\[Interruption\.\]',
               '<em>','</em>',r'\[Hon\. Members: "[\W\w]{1,28}"\]','\[[A-Za-z0-9\.;\s,\-]{1,65}\]','\[R\] '
                , #for training - remember that the first version didn't have a '-'
                '\[','\]']
    
    for cm in complexr:
        x = re.sub(cm,' ',x) #repeated space to space
        cleanups_log.append([cm,' '])
    
    x = re.sub('\s\s+',' ',x) #repeated space to space
    x = re.sub('^\s+','',x) #leading space
    x = re.sub('^Q[0-9]+\.','',x) #leading Q with number for the questions
    x = re.sub(' \([0-9]+\)\s*','',x) #question entries brackets
    x = re.sub(' \[[0-9]+\]\s*','',x) #question entries square brackets
    x = re.sub('\s+$','',x) #trailing space
    x = re.sub('[<>]','',x) #triangular brackets
    x = re.sub('^[0-9]{1,2}\.','',x) #leading number at start of question with no Q 
    return x,cleanups_log





In [20]:
regs = '\[[A-Za-z0-9\.;\s,\-]{1,65}\]'
testa = '[Official Report, 1 December 2021; Vol. 704, c. 909.]'
testt = '[Official Report, 8 December 2021; Vol. 705, c. 499-500.]'

print(re.search(regs,testt))

<re.Match object; span=(0, 57), match='[Official Report, 8 December 2021; Vol. 705, c. 4>


— : clause indicator - replace with space.
-: word joiner - leave alone


Do a check for child debates within the child debates item of the json boject, and for contributions outslide of the child debates item.

In [127]:
s='—'
s.encode("unicode_escape") #include this in text cleaning section.


b'\\u2014'

In [18]:
testrep.replace("\u2014","*")

'data-driven help*me if you can*help'

In [6]:
#appendlist[0]['ChildDebates']

In [81]:
print('\x96')
“




https://www.codetable.net/hex/96 - checks unicode hex character codes, which look like \x96 in Python
https://www.compart.com/en/unicode/html - check for unicode character codes, which look like \u2014 in python

In [166]:
list(re.finditer(r'\[Hon\. Members: "[\W\w]{1,28}"\]','[Hon. Members: "Hear, hear."]'))[0][0]

'[Hon. Members: "Hear, hear."]'

In [11]:

#FINAL

#mp_ids = []
for c in contribs:
    cleaned,reps = cleanup(c[4])
    c.append(cleaned)
    c.append(reps)
    
    #mp_ids.append(c[1])
    

In [12]:
#FINAL


allmps = list(set([c[1] for c in contribs if c[1] is not None]))

In [13]:
#FINAL


mpqs = "https://hansard-api.parliament.uk/search/members.json?queryParameters.house=Commons&queryParameters.memberId={0}"

In [192]:
#Original info scrape - do not run. Run file import instead


mp_data_list = {}
for mp in allmps:
    qrs = mpqs.format(mp)
    q = requests.get(qrs)
    mpjson = json.loads(q.text)
    mpdict = mpjson['Results'][0]
    mp_data = [mpdict['DisplayAs'],mpdict['Party'],mpdict['MemberFrom']]
    mp_data_list[mp] = mp_data
    sleep(1)

In [198]:
parties = [mp_data_list[mp][1] for mp in mp_data_list]

In [14]:
#FINAL


role_override = {#1423:['PM','Con','Boris Johnson'],
            #4390:['ThirdPartyLead','SNP','Ian Blackford'],
            1510:['Dep LOE (replacing)','Lab','Ed Miliband'],
            4356:['Dep LOE (replacing)','Lab','Angela Rayner'],
            #4007:['Dep PM (replacing)','Con','Dominic Raab'],
            4441:['Dep PM (replacing)','Con','Oliver Dowden'],
            4514:['LOE','Lab','Kier Starmer'],
            467:['Speaker','Speakers','Lindsay Hoyle'],
            36:['Deputy Speaker','Speakers','Eleanor Laing'],
            #4097:['PM','Con','Elizabeth Truss'],
            4483:['PM','Con','Rishi Sunak'],
            4735:['ThirdPartyLead','SNP','Stephen Flynn']
}




partytext = {'Alba Party':'Alba (Sco)',
 'Alliance':'Alliance (NI)',
 'Conservative':'Con',
 'Democratic Unionist Party':'DUP',
 'Green Party':'Green',
 'Independent':'Independent',
 'Labour':'Lab',
 'Labour (Co-op)':'Lab',
 'Liberal Democrat':'LD',
 'Plaid Cymru':'Plaid Cymru',
 'Scottish National Party':'SNP',
 'Social Democratic & Labour Party':'SDLP',
 'Speaker':'Speakers'}

In [215]:
for m in mp_data_list:
    if m in role_override:
        mp_data_list[m][0] = role_override[m][2]
        mp_data_list[m][1] = role_override[m][1]
        
        mp_data_list[m].append(role_override[m][0])
        print(mp_data_list[m])
    else:
        mp_data_list[m].append('BB')

['Angela Rayner', 'Lab', 'Ashton-under-Lyne', 'Dep LOE (replacing)']
['Ian Blackford', 'SNP', 'Ross, Skye and Lochaber', 'ThirdPartyLead']
['Boris Johnson', 'Con', 'Uxbridge and South Ruislip', 'PM']
['Kier Starmer', 'Lab', 'Holborn and St Pancras', 'LOE']
['Dominic Raab', 'Con', 'Esher and Walton', 'Dep PM (replacing)']
['Lindsay Hoyle', 'Speakers', 'Chorley', 'Speaker']
['Ed Miliband', 'Lab', 'Doncaster North', 'Dep LOE (replacing)']


In [15]:
#FINAL
_pmqmaster = "ALLCONTRIBSMASTER -CAT.txt" # ROlling raw transcripts Database file
_catmps = "MPRolling.txt" # Rolling lookup file of MPs with party info.

In [224]:
#with open(_catmps,'w',newline='') as mpf:
#    wrt = csv.writer(mpf, delimiter='\t')
#    wrt.writerow(['MPID','Name','Party','Constituency','Role'])
#    for m in mp_data_list:
#        rw = [m]
#        rw.extend(mp_data_list[m])
#        wrt.writerow(rw)

In [16]:
#FINAL


#read catfile into dictionary
mp_data_list = {}

with open(_catmps,'r',newline='') as mpf:
    mpf.readline()
    rdr = csv.reader(mpf, delimiter='\t')
    
    for r in rdr:
        mp_data_list[int(r[0])] = r[1:]

In [17]:
#FINAL
mps_not_in_list = list(set(allmps).difference(set([m for m in mp_data_list])))

In [18]:
mps_not_in_list

[1440, 4066, 3945, 4682, 429, 3919, 1585]

In [21]:
qrs = mpqs.format(4483)
qrs
json.loads(requests.get(qrs).text)

{'Results': [{'MemberId': 4483,
   'DodsId': 136507,
   'PimsId': 6165,
   'DisplayAs': 'Rishi Sunak',
   'ListAs': 'Sunak, Rishi',
   'FullTitle': 'Rt Hon Rishi Sunak MP',
   'LayingMinisterName': 'The Prime Minister',
   'HistoricalMemberName': None,
   'HistoricalFullTitle': None,
   'Gender': 'M',
   'Party': 'Conservative',
   'PartyId': 4,
   'House': 'Commons',
   'MemberFrom': 'Richmond (Yorks)',
   'HouseStartDate': '2015-05-07T00:00:00',
   'HouseEndDate': None,
   'IsTeller': False,
   'SortOrder': 0,
   'ConstituencyCountry': 'E'}],
 'TotalResultCount': 1}

In [19]:
#FINAL


#new mps from latest scrape - would run an API call against this,
if mps_not_in_list:
    
    # do the calls and add to new dict list
    mp_data_list_n = {}
    for mp in mps_not_in_list:
        qrs = mpqs.format(mp)
        q = requests.get(qrs)
        mpjson = json.loads(q.text)
        mpdict = mpjson['Results'][0]
        mp_data = [mpdict['DisplayAs'],mpdict['Party'],mpdict['MemberFrom']]
        mp_data_list_n[mp] = mp_data
        sleep(1)
    #update parties
    for m in mp_data_list_n:
        mp_data_list_n[m][1] = partytext[mp_data_list_n[m][1]]
    # roles and specific member exceptions
    for m in mp_data_list_n:
        if m in role_override:
            mp_data_list_n[m][0] = role_override[m][2]
            mp_data_list_n[m][1] = role_override[m][1]
            
            mp_data_list_n[m].append(role_override[m][0])
            print(mp_data_list_n[m])
        else:
            mp_data_list_n[m].append('BB')
    # add them to the existing list
    for m in mp_data_list_n:
        mp_data_list[m] = mp_data_list_n[m]
    # rewrite mp lookup file to reflect updates.   
    with open(_catmps,'w',newline='') as mpf:
        wrt = csv.writer(mpf, delimiter='\t')
        wrt.writerow(['MPID','Name','Party','Constituency','Role'])
        for m in mp_data_list:
            rw = [m]
            rw.extend(mp_data_list[m])
            wrt.writerow(rw)

In [20]:
#FINAL


with open(_pmqmaster,'a',newline='') as fl:
    wrt = csv.writer(fl,delimiter='\t')


    for cn in contribs:
        if cn[1] is not None:
        
    
            cnrow = [cleanup(cn[2])[0], #unrefined text name
             '',
             '/search/MemberContributions?house=Commons&memberId={0}'.format(cn[1]), # link for mp
             cn[5], #clean text
             cn[0], #date
             mp_data_list[cn[1]][3], #role
             mp_data_list[cn[1]][1], #party
             mp_data_list[cn[1]][0], #name
            ]
            wrt.writerow(cnrow)